In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import OrderedDict
import pandas as pd
import numpy as np
from recs.jaxmodels.cf import bprmf
from recs.evaluator import evaluate
from recs import utils
from recs.dataset import negative_sample_dataset

import tensorflow as tf
from tqdm.notebook import tqdm

import jax
from jax import numpy as jnp

In [ ]:
train_data, train_len, num_items, num_users = negative_sample_dataset(batch_size=500)

In [ ]:
num_items, num_users, train_len

In [ ]:
params = bprmf.init_params(
    num_items,
    num_users,
    30
)

In [ ]:
losses = []
for epoch in range(10):
    batch_loss = 0.
    with tqdm(train_data.as_numpy_iterator(), desc=f"[Epoch {epoch+1}]", total=train_len) as tm:
        for i, batch in enumerate(tm):
            params, error = bprmf.update(params, batch, alpha=0.005, lam=0.05)
            batch_loss += error
            tm.set_postfix(OrderedDict(loss=batch_loss / (i + 1)))
    batch_loss /= train_len
    losses += np.asarray(batch_loss)

In [ ]:
def predict_items(params, data, k):
    users = np.unique(data[:, 0])
    pred_dic = {}
    for u in users:
        pred_dic[u] = np.asarray(mf.predict(params, u, k))
    return pd.DataFrame(pred_dic.items(), columns=["userId", "itemIds"])early_stop_count